In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import matplotlib.pyplot as plt
from math import log, sqrt
import pandas as pd
import numpy as np
import nltk

In [2]:
mails = pd.read_csv('spam.csv', encoding = 'latin-1')
mails.head()

,Label,Message,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [3]:
mails = np.array(mails)
np.random.shuffle(mails)

In [4]:
mail = mails[:,[1]]

In [5]:
lable = mails[:,[0]]

In [6]:
t = mail.shape[0] * 0.7

In [7]:
t = int(t)

In [8]:
mail_train = mail[:t]
mail_test = mail[t:]

In [9]:
lable_train = lable[:t]
lable_test = lable[t:]

In [10]:
def process_message(message, lower_case = True, stem = True, stop_words = True, gram = 1):
    if lower_case:
        message = message.lower()
    words = word_tokenize(message)
    words = [w for w in words if len(w) > 2]
    if gram > 1:
        w = []
        for i in range(len(words) - gram + 1):
            w += [' '.join(words[i:i + gram])]
        return w
    if stop_words:
        sw = stopwords.words('english')
        words = [word for word in words if word not in sw]
    if stem:
        stemmer = PorterStemmer()
        words = [stemmer.stem(word) for word in words]   
    return words

In [11]:
spam_dict = {}
ham_dict = {}
def find_freq(X, Y):
    no_of_mails = X.shape[0]
    for i in range(no_of_mails):
        if Y[i][0] == 'spam':
            message_processed = process_message(X[i][0])
            for j in message_processed:
                spam_dict[j] = spam_dict.get(j, 0) + 1
        if Y[i][0] == 'ham':
            message_processed = process_message(X[i][0])
            for j in message_processed:
                ham_dict[j] = ham_dict.get(j, 0) + 1
find_freq(mail_train, lable_train)

In [12]:
prob_spam = {}
prob_ham = {}
spam_words = sum(spam_dict.values())
ham_words = sum(ham_dict.values())
spam_mails = 0
ham_mails = 0
for i in lable_train:
    if i == 'spam':
        spam_mails += 1
    else:
        ham_mails += 1
prob_spam_mails = 0
prob_ham_mails = 0
prob_spam_mails = spam_mails / (spam_mails + ham_mails)
prob_ham_mails = ham_mails / (ham_mails + spam_mails)

In [13]:
def find_prob(X, Y):
    for i in X:
        prob_spam[i] = (spam_dict[i] + 1)/ (spam_words + len(list(spam_dict.keys())))
    for i in Y:
        prob_ham[i] = (ham_dict[i] + 1)/ (ham_words + len(list(ham_dict.keys())))

In [14]:
find_prob(spam_dict, ham_dict)
prob_spam_mails

0.13128205128205128

In [15]:
prob_ham_mails

0.8687179487179487

In [16]:
def classify(mail_test, lable_test):
    no_of_mails = mail_test.shape[0]
    result = []
    for i in range(no_of_mails):
        message = process_message(mail_test[i][0])
        pspam = 0
        pham = 0
        for w in message:
            if w in prob_spam:
                pspam += log(prob_spam[w])
            else:
                pspam -= log(spam_words + len(list(spam_dict.keys())))
        for w in message:
            if w in prob_ham:
                pham += log(prob_ham[w])
            else:
                pham -= log(ham_words + len(list(ham_dict.keys())))
        pspam += log(prob_spam_mails)
        pham += log(prob_ham_mails)
        if (pspam >= pham):
            result.append('spam')
        else:
            result.append('ham')
    ans = 0
    for i in range(no_of_mails):
        if result[i] == lable_test[i][0]:
            ans+=1
    acc = (ans/no_of_mails) * 100
    return result, acc

In [17]:
result, acc = classify(mail_test, lable_test)

In [18]:
acc

94.31818181818183